In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.append( "../../lattice/")
sys.path.append("../")
sys.path.append( "../../impurity/")

from HC_Lattice import HC_Lattice
from DMET_solver import DMET_solver
from Impurity_solver import Impurity_solver
import matplotlib.pyplot as plt
import seaborn
import scipy

%matplotlib notebook


/home/romain/.conda/envs/DRL/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/romain/.conda/envs/DRL/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


../../mean_field/


In [2]:
def get_double_occupancy(Q, i):
    '''
    Compute <ni_up ni_down>
    :param: Q is a 2nxn matrix, whose columns are the occupied eigenstates of H_MF
    :param: i is an integer, corresponding to the site of interest
    :return: double_occupancy is the expected double occupancy on site i
    '''
    # Get the 2 rows of Q corresponding to site i
    n = int(len(Q[:, 0]) / 2)
    O = Q[[i, i + n], :].copy()
    # Perform the singular value decomposition of O
    U, s, Vh = scipy.linalg.svd(O)
    V = np.transpose(Vh)
    # S=np.zeros( [len(U),len(Vh)])
    # for i in np.arange(len(U)):
    #    S[i,i]=s[i]
    # print(np.matmul(np.matmul(U,S),Vh))
    # Perform the rotation
    Q_tilde = np.matmul(Q, V)
    # return the expected double occupancy
    return (Q_tilde[i, 1] * Q_tilde[i + n, 0] - Q_tilde[i + n, 1] * Q_tilde[i, 0]) ** 2


def get_mean_double_occupancy(Q, n_el):
    d = []
    for i in np.arange(int(n_el)):
        d.append(get_double_occupancy(Q, i))
    return np.mean(d)


def squared_staggered_magnetization(Q, n_el):
    rho = np.matmul(Q, np.conjugate(np.transpose(Q)))  # single particle density function
    mean_occ = np.diag(rho)
    m = 0
    n = int(n_el)

    for i in np.arange(n):
        [x, y] = lattice.find_site(i + 1)
        y = y % 2  # sublattice A is on odd columns, B on even
        signe = (-1) ** y[0]
        m += signe * (mean_occ[i] - mean_occ[i + n])
    return np.real(np.abs(m / n) ** 2)



In [3]:
#Create the lattice
ls=10
lattice = HC_Lattice(height=ls, length=ls)
# initialize DMET_solver
n=lattice.nb_sites
print(lattice.lattice)

t=1
u=t
lattice_size=[ls,ls]
#solve the system
dms = DMET_solver( t, u, n, lattice_size,[23,24],[23,24,28,18,19,29])


[[ 1  2  0  0  3  4  0  0  5  6]
 [ 0  0  7  8  0  0  9 10  0  0]
 [11 12  0  0 13 14  0  0 15 16]
 [ 0  0 17 18  0  0 19 20  0  0]
 [21 22  0  0 23 24  0  0 25 26]
 [ 0  0 27 28  0  0 29 30  0  0]
 [31 32  0  0 33 34  0  0 35 36]
 [ 0  0 37 38  0  0 39 40  0  0]
 [41 42  0  0 43 44  0  0 45 46]
 [ 0  0 47 48  0  0 49 50  0  0]]


In [4]:
n_points=10
D=np.zeros([n_points])
m=np.zeros([n_points])

t=1
for [i,u] in enumerate(np.linspace(0,10*t,n_points)):
    print("new u = "+str(u))
    dms = DMET_solver( t, u, n, lattice_size,[23,24],[23,24,28,18,19,29])
    n_el = dms.lattice.nb_sites
    dms.solve_system()
    Q = dms.get_mean_field_orbitals(dms.potential)
    D[i]=get_mean_double_occupancy(Q,n_el)
    m[i]=squared_staggered_magnetization(Q,n_el)

new u = 0.0
Last potential's move : 44.143363387111805
density dif : 0.28448375460006775
Last potential's move : 0.3158185250294183
density dif : 0.5121626279523104
Last potential's move : 0.33160945128088937
density dif : 0.5033282181488016
Last potential's move : 0.3481899238449335
density dif : 0.4942460357517945


KeyboardInterrupt: 

In [ ]:
list_u = np.linspace(0,10*t,n_points)
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(list_u, D, "b-o")
ax1.set_xlabel('U/t')
# Make the y-axis label, ticks and tick labels match the line color.
ax2.plot(list_u, m, "--o",color='r')

ax1.tick_params('y',colors='b')
ax1.set_ylabel('d', color='b')
ax2.set_ylabel('m^2', color='r')
ax2.tick_params('y', colors='r')

